## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2026-02-02-10-07-00 +0000,bbc,Lord Mandelson resigns from Labour Party over ...,https://www.bbc.com/news/articles/cn8jm2vpve1o...
1,2026-02-02-10-03-54 +0000,nyt,"After His Democratic Victory in Texas, a New W...",https://www.nytimes.com/2026/02/02/us/politics...
2,2026-02-02-10-03-53 +0000,nyt,Protesters Press Target to Take a Stand Agains...,https://www.nytimes.com/2026/02/02/us/target-p...
3,2026-02-02-10-02-49 +0000,nyt,Why The Times Is Expanding Its Supreme Court C...,https://www.nytimes.com/2026/02/02/us/supreme-...
4,2026-02-02-10-02-33 +0000,nyt,"Snow Drought, Political Shifts and Strikes Bat...",https://www.nytimes.com/2026/02/02/travel/snow...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2554/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
68,trump,25
20,ice,20
5,epstein,19
10,new,14
67,year,12


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
128,2026-02-01-22-34-48 +0000,nypost,India to buy oil from Venezuela rather than Ru...,https://nypost.com/2026/02/01/world-news/india...,67
92,2026-02-02-02-49-20 +0000,nypost,"5-year-old boy, father taken into custody by I...",https://nypost.com/2026/02/01/us-news/5-year-o...,64
90,2026-02-02-02-56-44 +0000,nyt,Trump Says Kennedy Center Will Close for 2-Yea...,https://www.nytimes.com/2026/02/01/us/politics...,62
49,2026-02-02-07-37-00 +0000,nypost,Donald Trump threatens to sue Trevor Noah afte...,https://nypost.com/2026/02/02/us-news/donald-t...,58
197,2026-02-01-15-09-48 +0000,nypost,Trump jokes about suing new Fed chair nominee ...,https://nypost.com/2026/02/01/us-news/trump-jo...,56


## Select top headlines

In [5]:
with open('exclude_sources.txt') as f:
    excluded_sources = {
        line.strip()
        for line in f
        if line.strip() and not line.strip().startswith('#')
    }

top_count = 10
top_rows = []
working = word_scores.copy()
remaining = latest.copy()

while len(top_rows) < top_count and not remaining.empty:
    scored_remaining = remaining.assign(
        score=remaining['title'].apply(
            lambda t: sum(
                working.get(w.lower(), 0)
                for w in re.findall(r'[A-Za-z]+', t)
                if len(w) > 1
            )
        )
    )
    ranked_loop = scored_remaining.sort_values('score', ascending=False)

    if ranked_loop.empty:
        break

    top_story = ranked_loop.iloc[0]
    remaining = remaining.drop(top_story.name)

    if top_story['source'] in excluded_sources:
        continue

    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for word in words:
        working.pop(word, None)

    top_rows.append(top_story[['score', 'pubdate', 'source', 'title', 'link']])

top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
128,67,2026-02-01-22-34-48 +0000,nypost,India to buy oil from Venezuela rather than Ru...,https://nypost.com/2026/02/01/world-news/india...
92,64,2026-02-02-02-49-20 +0000,nypost,"5-year-old boy, father taken into custody by I...",https://nypost.com/2026/02/01/us-news/5-year-o...
211,49,2026-02-01-13-52-32 +0000,nypost,Ex-Prince Andrew’s creepy reply to Jeffrey Eps...,https://nypost.com/2026/02/01/world-news/ex-pr...
233,41,2026-02-01-10-54-14 +0000,nypost,Iran’s supreme leader warns any US attack woul...,https://nypost.com/2026/02/01/world-news/iran-...
104,36,2026-02-02-01-31-39 +0000,nypost,Mike Johnson ‘confident’ gov’t shutdown will e...,https://nypost.com/2026/02/01/us-news/mike-joh...
81,31,2026-02-02-03-38-34 +0000,startribune,The latest: Musicians call out ICE actions in ...,https://www.startribune.com/ice-raids-minnesot...
44,28,2026-02-02-08-07-35 +0000,nyt,"Rafah Crossing in Gaza Reopens, Another Step f...",https://www.nytimes.com/2026/02/02/world/middl...
203,27,2026-02-01-14-26-13 +0000,cbc,More than a dozen people killed in Ukraine aft...,https://www.cbc.ca/news/world/ukraine-talks-ru...
132,27,2026-02-01-22-21-00 +0000,wsj,A state-level Democratic win in a region that ...,https://www.wsj.com/politics/elections/texas-e...
133,26,2026-02-01-22-15-31 +0000,bbc,'Do you think you're the devil himself?' Epste...,https://www.bbc.com/news/videos/cx20zqw7v9ko?a...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
